# Present Value Correction Derivation

**Created**: 2025-12-05  
**Purpose**: Document and verify the risk-neutral PV formula correction for FIA/RILA pricing

## The Bug

The original implementation used:
```python
PV = premium + discount_factor * premium * expected_credit
```

This incorrectly left the principal undiscounted while only discounting the credit.

## The Correct Formula [T1]

In a risk-neutral framework, we discount the **full maturity payoff**:

$$\text{PV} = e^{-rT} \times \text{Premium} \times (1 + \text{Expected Credit})$$

At maturity, the policyholder receives: `Premium * (1 + Expected Credit)`
Today's value is that payoff discounted to present.

## Impact Analysis

The bug overstated PV by approximately:
$$\text{Error} = \text{Premium} \times (1 - e^{-rT})$$

For r=5%, T=5 years, Premium=$100,000:
- Error ≈ $100,000 × (1 - e^{-0.25}) ≈ $22,100 (22% overstatement!)

In [ ]:
import numpy as np
import pandas as pd

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

## 1. Numerical Verification of Formula Difference

In [ ]:
def old_pv_formula(premium, rate, term, expected_credit):
    """OLD (incorrect): Principal not discounted."""
    df = np.exp(-rate * term)
    return premium + df * premium * expected_credit

def new_pv_formula(premium, rate, term, expected_credit):
    """NEW (correct): Full payoff discounted."""
    df = np.exp(-rate * term)
    return df * premium * (1 + expected_credit)

# Test cases
test_cases = [
    {"premium": 100_000, "rate": 0.05, "term": 1, "expected_credit": 0.03},
    {"premium": 100_000, "rate": 0.05, "term": 5, "expected_credit": 0.05},
    {"premium": 100_000, "rate": 0.05, "term": 10, "expected_credit": 0.08},
    {"premium": 100_000, "rate": 0.03, "term": 6, "expected_credit": 0.04},  # RILA typical
]

results = []
for tc in test_cases:
    old_pv = old_pv_formula(**tc)
    new_pv = new_pv_formula(**tc)
    error = old_pv - new_pv
    error_pct = error / new_pv * 100
    
    results.append({
        "Term (Y)": tc["term"],
        "Rate": tc["rate"],
        "Credit": tc["expected_credit"],
        "Old PV": f"${old_pv:,.0f}",
        "New PV": f"${new_pv:,.0f}",
        "Error": f"${error:,.0f}",
        "Error %": f"{error_pct:.1f}%"
    })

pd.DataFrame(results)

## 2. Verification with Actual Pricers

Now verify the corrected FIA and RILA pricers produce sensible values.

In [ ]:
from annuity_pricing.data.schemas import FIAProduct, RILAProduct
from annuity_pricing.products.fia import FIAPricer, MarketParams as FIAMarketParams
from annuity_pricing.products.rila import RILAPricer, MarketParams as RILAMarketParams

In [ ]:
# Standard market parameters
fia_params = FIAMarketParams(
    spot=100.0,
    risk_free_rate=0.05,
    dividend_yield=0.02,
    volatility=0.20,
)

rila_params = RILAMarketParams(
    spot=100.0,
    risk_free_rate=0.05,
    dividend_yield=0.02,
    volatility=0.20,
)

fia_pricer = FIAPricer(market_params=fia_params, seed=42)
rila_pricer = RILAPricer(market_params=rila_params, seed=42)

In [ ]:
# Test FIA with 10% cap
fia_product = FIAProduct(
    company_name="Test Life",
    product_name="S&P 500 Cap",
    product_group="FIA",
    status="current",
    cap_rate=0.10,
    index_used="S&P 500",
)

fia_result = fia_pricer.price(fia_product, premium=100_000, term_years=1.0)

print("FIA Pricing (10% Cap, 1Y Term)")
print(f"  Premium:        ${100_000:,.0f}")
print(f"  Expected Credit: {fia_result.expected_credit:.2%}")
print(f"  Present Value:  ${fia_result.present_value:,.0f}")
print(f"  Discount Factor: {np.exp(-0.05 * 1.0):.4f}")

# Verify formula
expected_pv = np.exp(-0.05 * 1.0) * 100_000 * (1 + fia_result.expected_credit)
print(f"  Formula Check:  ${expected_pv:,.0f}")
print(f"  Match: {abs(fia_result.present_value - expected_pv) < 0.01}")

In [ ]:
# Test RILA with 10% buffer, 15% cap
rila_product = RILAProduct(
    company_name="Test Life",
    product_name="10% Buffer S&P",
    product_group="RILA",
    status="current",
    buffer_rate=0.10,
    buffer_modifier="Losses Covered Up To",
    cap_rate=0.15,
    index_used="S&P 500",
)

rila_result = rila_pricer.price(rila_product, premium=100_000, term_years=1.0)

print("\nRILA Pricing (10% Buffer, 15% Cap, 1Y Term)")
print(f"  Premium:         ${100_000:,.0f}")
print(f"  Expected Return: {rila_result.expected_return:.2%}")
print(f"  Present Value:   ${rila_result.present_value:,.0f}")
print(f"  Discount Factor: {np.exp(-0.05 * 1.0):.4f}")

# Verify formula
expected_pv = np.exp(-0.05 * 1.0) * 100_000 * (1 + rila_result.expected_return)
print(f"  Formula Check:   ${expected_pv:,.0f}")
print(f"  Match: {abs(rila_result.present_value - expected_pv) < 0.01}")

## 3. Multi-Term Analysis

Show that PV behaves correctly across different terms.

In [ ]:
terms = [1, 2, 3, 5, 7, 10]
fia_results = []

for t in terms:
    result = fia_pricer.price(fia_product, premium=100_000, term_years=float(t))
    df = np.exp(-0.05 * t)
    
    fia_results.append({
        "Term (Y)": t,
        "Discount Factor": f"{df:.4f}",
        "Expected Credit": f"{result.expected_credit:.2%}",
        "Present Value": f"${result.present_value:,.0f}",
        "PV as % of Premium": f"{result.present_value / 100_000:.1%}"
    })

print("FIA Present Value by Term (10% Cap)")
pd.DataFrame(fia_results)

## 4. Sanity Checks [T1]

1. **Zero Credit**: PV = e^(-rT) × Premium (just discounted principal)
2. **Zero Rate**: PV = Premium × (1 + Credit) (no discounting)
3. **PV ≤ Premium × (1 + Credit)**: Always true with positive rates

In [ ]:
# Sanity Check 1: Zero credit should give just discounted principal
def check_zero_credit():
    premium, rate, term = 100_000, 0.05, 5
    pv = new_pv_formula(premium, rate, term, 0.0)
    expected = premium * np.exp(-rate * term)
    print(f"Zero Credit Check:")
    print(f"  PV:       ${pv:,.2f}")
    print(f"  Expected: ${expected:,.2f}")
    print(f"  Match: {abs(pv - expected) < 0.01}")
    return abs(pv - expected) < 0.01

# Sanity Check 2: Zero rate should give undiscounted maturity value
def check_zero_rate():
    premium, rate, term, credit = 100_000, 0.0, 5, 0.10
    pv = new_pv_formula(premium, rate, term, credit)
    expected = premium * (1 + credit)
    print(f"\nZero Rate Check:")
    print(f"  PV:       ${pv:,.2f}")
    print(f"  Expected: ${expected:,.2f}")
    print(f"  Match: {abs(pv - expected) < 0.01}")
    return abs(pv - expected) < 0.01

# Sanity Check 3: PV should be less than maturity value when rate > 0
def check_pv_less_than_maturity():
    premium, rate, term, credit = 100_000, 0.05, 5, 0.10
    pv = new_pv_formula(premium, rate, term, credit)
    maturity_value = premium * (1 + credit)
    print(f"\nPV < Maturity Value Check:")
    print(f"  PV:             ${pv:,.2f}")
    print(f"  Maturity Value: ${maturity_value:,.2f}")
    print(f"  PV < Maturity:  {pv < maturity_value}")
    return pv < maturity_value

all_pass = check_zero_credit() and check_zero_rate() and check_pv_less_than_maturity()
print(f"\n{'='*50}")
print(f"All sanity checks passed: {all_pass}")

## 5. Summary

### Key Findings

1. **Bug Impact**: The original formula overstated PV by ~22% for 5-year products at 5% rate
2. **Correction**: `PV = e^(-rT) × Premium × (1 + Expected Credit)` is the correct risk-neutral formula
3. **Fair Terms Unaffected**: `_solve_fair_cap` and `_solve_fair_participation` use BS option values (correctly discounted internally)

### Files Modified

- `src/annuity_pricing/products/fia.py:197-201` - FIA PV formula
- `src/annuity_pricing/products/rila.py:206-211` - RILA PV formula

### Test Updates

Tests in `tests/unit/test_products_fia.py` and `tests/unit/test_products_rila.py` continue to pass because:
1. They test relative relationships (higher cap → higher credit), not absolute PV values
2. The expected_credit calculation is unchanged
3. Only the final PV discounting was fixed